In [ ]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [1]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [3]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [4]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [5]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_A3_015' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A3_015
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.02064539328365893
test_rmse: 0.02057163385429561
test_r2: 0.5488546356466162
feature number: 3
train_rmse: 0.02048999189083751
test_rmse: 0.020646830496446927
test_r2: 0.5454241152686798
feature number: 4
train_rmse: 0.02031027295821894
test_rmse: 0.02066060888839557
test_r2: 0.5440987567885944
feature number: 5
train_rmse: 0.020239521838189722
test_rmse: 0.0207523945013787
test_r2: 0.5402381615311772
feature number: 6
train_rmse: 0.01978911645958172
test_rmse: 0.02063523156710085
test_r2: 0.5449418711182984
feature number: 7
train_rmse: 0.019713512282696334
test_rmse: 0.020504559214319984
test_r2: 0.5508460551254091
feature number: 8
train_rmse: 0.019697876881672422
test_rmse: 0.020606791746894237
test_r2: 0.5459749727797202
feature number: 9
train_rmse: 0.019671607675643823
test_rmse: 0.020617834704631695
test_r2: 0.5455964236668807
feature number: 10
train_rmse: 0.01965123887015217
test_rmse: 0.02063557865750825


train_rmse: 0.019477377579606228
test_rmse: 0.020859341865798593
test_r2: 0.5354078488707311
feature number: 17
train_rmse: 0.019462327712310067
test_rmse: 0.020942476496550046
test_r2: 0.5315957150493913
feature number: 18
train_rmse: 0.01944112874879887
test_rmse: 0.020985814440144253
test_r2: 0.5295635182626968
feature number: 19
train_rmse: 0.019431672166018713
test_rmse: 0.02102612310831662
test_r2: 0.5278238369126019
feature number: 20
train_rmse: 0.019406460077280648
test_rmse: 0.021173288802769683
test_r2: 0.5210628377557469
Test_rmse_min = 1.fea_num: 7  2.rmse: 0.020504559214319984
Test_r2_max = 1. fea_num: 7  2.r2: 0.5508460551254091
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.02064539328365893
test_rmse: 0.02057163385429561
test_r2: 0.5488546356466162
feature number: 3
train_rmse: 0.02048999189083751
test_rmse: 0.020646830496446927
test_r2: 0.5454241152686798
feature number: 4
train_rmse: 0.02031027295821894
test_rmse: 0.02066060888839557
test_r2: 0.54409875678

train_rmse: 0.01949721336590824
test_rmse: 0.020540420104053245
test_r2: 0.5493681554966775
feature number: 12
train_rmse: 0.01948706455640978
test_rmse: 0.020587742181010244
test_r2: 0.5472252918348148
feature number: 13
train_rmse: 0.019465776864218454
test_rmse: 0.020639070767437935
test_r2: 0.5449864742820056
feature number: 14
train_rmse: 0.019462722159233104
test_rmse: 0.02066008262568533
test_r2: 0.543899663678573
feature number: 15
train_rmse: 0.019456143208184295
test_rmse: 0.020682479520407752
test_r2: 0.5429980175378308
feature number: 16
train_rmse: 0.019440424887048755
test_rmse: 0.02073784673971624
test_r2: 0.5406556995301686
feature number: 17
train_rmse: 0.019390705760938022
test_rmse: 0.020816298001527
test_r2: 0.5373194835643836
feature number: 18
train_rmse: 0.01938157491529483
test_rmse: 0.020847991714527822
test_r2: 0.5359663638819232
feature number: 19
train_rmse: 0.01936908455617037
test_rmse: 0.020921511157698535
test_r2: 0.5326048742021077
feature number: 20
tr

# 可以修改第一層的神經元數量

In [11]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(5, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [8]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量 神經元5

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_0/Input_A3_015.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.02061778204964424
[0.016992046644737725, 0.022558195356958705, 0.01828758376236964, 0.02238138115989026, 0.018602054286224405, 0.01827015618429703, 0.02138242281477221, 0.02148384614006769, 0.025549941652979243, 0.020670192494145492]


test_r2_ave: 0.5475587748869882
[0.7032439120588574, 0.483974347533913, 0.6347383780784249, 0.4312977523096512, 0.6232610534915086, 0.6001161574678908, 0.5654698848919548, 0.5522684544208124, 0.40935414149776994, 0.4718636671190981]


feature number: 6
test_rmse_ave: 0.021597065694440233
[0.019339732433407062, 0.02501743005305825, 0.020532638646382745, 0.021099235384324056, 0.01960832044016284, 0.019288449968575525, 0.024382653317284095, 0.023758805638204303, 0.018952408642765133, 0.023990982420238376]


test_r2_ave: 0.5001687974171392
[0.6155771654403681, 0.36532999562205526, 0.5395516566763325, 0.4945890903670791, 0.5813997385468015, 0.554298603910496, 0.4349745861806432, 0.4524259770313275, 0.6750053876476266, 0.2885